# Stack und Unstack

- Funktionsweise
- Erzeugen und Rücksetzen eines Multiindex
- Datentyp für Perioden (fortlaufende Monate)

Datenquelle: https://www.dwd.de/DE/leistungen/zeitreihen/zeitreihen.html

In [ ]:
import pandas as pd
import numpy as np

Test mit einem DataFrame

In [ ]:
df = pd.read_csv('temp-all.csv')
df

Aufruf von `stack()` stapelt alle Spalten, auch das Jahr

In [ ]:
x = df.stack()
x

In [ ]:
x.info()

Spalten, die nicht gestapelt werden sollen, nehmen wir in den Index auf

In [ ]:
x2 = df.set_index('year')
#df.info()
x2.info()

In [ ]:
x3 = x2.stack()
x3.info()

Rücksetzen des Index, wird wieder normale Spalte.

Meist ist ein Umbenennen der Spalten sinnvoll.

In [ ]:
x4 = x3.reset_index()
x4.columns = ['year', 'month', 'val']
x4

Hilfsfunktion zum Einlesen der 3 Data Frames, Hinzufügen einer Spalte `feature`, um die verschiedenen Messwerte unterscheiden zu können.

In [ ]:
def read_addfeature(file_name, feature_name):
    df = pd.read_csv(file_name)
    df['feature'] = feature_name
    print(df.head()) # auskommentieren!
    return df

Einlesen der 3 Data Frames

In [ ]:
df_temp = read_addfeature('temp-all.csv', 'temperature')
df_rain = read_addfeature('nied-all.csv', 'rainfall')
df_sun = read_addfeature('son-all.csv', 'sunshine')

Vertikales Zusammenfügen der Data Frames. Spaltenreihenfolge und -namen sind identisch.

In [ ]:
data = pd.concat([df_temp, df_rain, df_sun])
data

Bilden des Multiindex. Feature und Jahr sollen nicht gestapelt werden.

Entscheidung über Reihenfolge ist zu treffen:
- Erst das Feature, dann zu diesem alle Jahre zuordnen
- Erst das Jahr, zu diesem die 3 Features zuordnen

In [ ]:
data_idx = data.set_index(['feature', 'year'])
print(data_idx.info())
data_idx

Stapeln der Werte. Alle Messwerte stehen nun in einer Spalte

In [ ]:
data_stacked = data_idx.stack()
data_stacked.head(20)

Index wird zurückgesetzt, Vergabe sinnvoller Spaltennamen

In [ ]:
data_flat = data_stacked.reset_index()
data_flat.columns = ['feature', 'year', 'month', 'value']
data_flat

Die 3 Ausprägungen des Features (Temp., Niederschlag, Sonne) sollen jeweils eine eigene Spalte bilden.

Im Multiindex muss `feature` an letzter Stelle stehen. Für die korrekte zeitliche Reihenfolge muss im Index zuerst das Jahr, dann der Monat kommen.

In [ ]:
data_for_unstack = data_flat.set_index(['year', 'month', 'feature'])
data_for_unstack

»Entstapeln« der Feature-Ausprägungen zu eigenen Spalten

In [ ]:
data_unstacked = data_for_unstack.unstack()
data_unstacked

Rücksetzen des Multiindex, Vergabe sinnvoller Spaltennamen

In [ ]:
observations = data_unstacked.reset_index()
observations.columns = ['year', 'month', 'rainfall', 'sunshine', 'temperature']
observations

Kontrolle der Struktur des Data Frame, Datentyp für Monat ist ungünstig, besser `int`

In [ ]:
observations.info()

Typumwandlung für Monat

In [ ]:
observations['month'] = observations['month'].astype(int)
print(observations.info())
observations

Erzeugen von Jahr-Monat-Strings aus den Spalten `year` und `month`

- Achse muss angegeben werden
- Typ wird nicht korrekt erkannt, Umwandlung mit `int()` erzwingen

In [ ]:
y_m = observations.apply(lambda row: '{:04}-{:02}'.format(int(row['year']), int(row['month'])), axis=1)
y_m                     

Erzeugen einer Spalte vom Typ `Period` für Zeitreihenanalysen

In [ ]:
observations['period'] = pd.PeriodIndex(y_m, freq='M')
print(observations.info())
observations

Der `Period`-Typ kann einfach in einen String umgewandelt werden.

In [ ]:
observations['period'].astype(str)

Entfernen von Zeilen (`axis=0`) bzw. Spalten (`axis=1`), die fehlende Werte enthalten.

In [ ]:
observations.dropna(axis=0)

In [ ]:
observations.dropna(axis=1)

Umsortieren der Spalten mit Hilfe von `set_index()` und `reset_index()`

In [ ]:
obs_ordered = observations.set_index(['period', 'year', 'month']).reset_index()
obs_ordered

Danke für die Aufmerksamkeit!